In [1]:
#Import the following Libraries for Twitter Streaming and Analysis
# If any library fails to import, make sure that you have installed the libraries properly
import tweepy
import twitter
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
from tweepy import OAuthHandler
import json
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import operator 
import json
from collections import Counter
from collections import defaultdict
import sys
import os
import vincent
import matplotlib.pyplot as plt
import numpy as np
import pandas

# To run this code, first edit config.py with your configuration directory and then import the config to be called leater on
# This config.py will contain the follwoing information for a specific user

# consumer_key = ''
# consumer_secret = ''
# access_token = ''
# access_secret = ''

import config

In [2]:
import os
os.getcwd()

'C:\\Users\\Mridul\\Mining-the-Social-Web-2nd-Edition-master\\ipynb'

In [3]:
#Using config.py file imported earlier, authenticate and initialize the api's
#Below i have used two APIs "tweepy and Twitter", and initialized both of them
auth = OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_secret)
api = tweepy.API(auth)

oauth = twitter.oauth.OAuth(config.access_token, config.access_secret,config.consumer_key, config.consumer_secret)
twitter_api = twitter.Twitter(auth=oauth)

In [42]:
# Choose a file name for saving the Tweets
fname_save = 'CARvsDAL6.json'
#Choose a file name for analyzing the tweets
fname = 'CARvsDAL2.json'

In [ ]:
#Create a MyListener class and call the funtion on_data to collect a live stream of data 
# with a specific KeyWord for example #CARvsDAL

class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open(fname_save, 'a+') as f:
                f.write(data)
                f.close()
                
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#CARvsDAL'])

In [29]:
#Function to pretty print the JSON file we have collected in the previous function call
#this function is just for testing purposes to see that the JSON data is loaded from the file

with open(fname, 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    print(json.dumps(tweet, indent=4)) # pretty-print
    f.close()

{
    "contributors": null, 
    "truncated": false, 
    "text": "RT @Panthers: Who wants to name this dance because we have no idea. \n\n#Panthers up 30-6! #CARvsDAL https://t.co/nnAN6mvJAM", 
    "is_quote_status": false, 
    "in_reply_to_status_id": null, 
    "id": 670029255082942464, 
    "favorite_count": 0, 
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 
    "retweeted": false, 
    "coordinates": null, 
    "timestamp_ms": "1448582397610", 
    "entities": {
        "user_mentions": [
            {
                "id": 56443153, 
                "indices": [
                    3, 
                    12
                ], 
                "id_str": "56443153", 
                "screen_name": "Panthers", 
                "name": "Carolina Panthers"
            }
        ], 
        "symbols": [], 
        "hashtags": [
            {
                "indices": [
                    70, 
                    79
         

In [30]:
# Function to Pre-process the tweets
# @-mentions, emoticons, URLs and #hash-tags are now preserved as individual tokens
# We try to capture some emoticons, HTML tags, Twitter @usernames (@-mentions), 
# Twitter #hashtags, URLs, numbers, words with and without dashes and apostrophes, and finally “anything else”


emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [43]:
with open(fname, 'r') as f:
    for line in f:
        try:
            tweet = json.loads(line)
            tokens = preprocess(tweet['text'])
        except:
            pass

In [44]:
#"text":"RT @NFL: .@LukeKuechly Had a Day! #CARvsDAL https:\/\/t.co\/MngpsMwpfz"

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']
terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
print terms_stop

[u'RT', u'@dallascowboys', u'@ninjafast22', u'see', u'#CARvsDAL', u'https://t.co/3MGEYRWqSI']


In [45]:
''' 
We can further customise the list of terms/tokens we are interested in. For example:
We are interested in only All the Terms, #Terms, Terms without #/@ etc
'''

with open(fname, 'r') as f:
    count_all = Counter()
    count_all_hash = Counter()
    count_all_terms = Counter()
    for line in f:
        try: 
            tweet = json.loads(line)
            # Create a list with all the terms
            terms_all = [term for term in preprocess(tweet['text']) if term not in stop]
            # Update the counter
            count_all.update(terms_all)
            # Count terms only once, equivalent to Document Frequency
            terms_single = set(terms_all)
            # Count hashtags only
            terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('#')]
            count_all_hash.update(terms_hash)
            count_all.update(terms_hash)
            # Count terms only (no hashtags, no mentions)
            terms_only = [term for term in preprocess(tweet['text']) if term not in stop and not term.startswith(('#', '@'))] 
            count_all_terms.update(terms_only)
            count_all_terms.update(terms_only)
        except:
            pass
        
    # Print the first 5 most frequent words
    print(count_all.most_common(10))
    print(count_all_hash.most_common(10))
    print(count_all_terms.most_common(10))

[(u'#CARvsDAL', 46920), (u'RT', 18294), (u'@Panthers', 8736), (u'#LUUUUKE', 7560), (u'Romo', 4781), (u'@NFL', 4551), (u'Tony', 3942), (u'#Panthers', 3886), (u'Luke', 3645), (u'\ud83d', 3593)]
[(u'#CARvsDAL', 23460), (u'#LUUUUKE', 3780), (u'#Panthers', 1943), (u'#Pick6', 834), (u'#Cowboys', 734), (u'#KeepPounding', 593), (u'#NFL', 233), (u'#CowboysNation', 224), (u'#NFLThanksgiving', 181), (u'#Thanksgiving', 167)]
[(u'RT', 36588), (u'Romo', 9562), (u'Tony', 7884), (u'Luke', 7290), (u'\ud83d', 7186), (u'Kuechly', 7130), (u'Another', 7100), (u'interception', 7100), (u"Don't", 6880), (u'test', 6838)]


In [46]:
with open(fname, 'r') as f:
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        try:
            terms_all = [term for term in preprocess(tweet['text']) if term not in stop]
        except:
            pass

In [47]:
com = defaultdict(lambda : defaultdict(int))
with open(fname, 'r') as f:
    # f is the file pointer to the JSON data set
    for line in f: 
        tweet = json.loads(line)
        try:
            terms_only = [term for term in preprocess(tweet['text']) 
                      if term not in stop 
                      and not term.startswith(('#', '@'))]
        except:
            pass
        # Build co-occurrence matrix
        for i in range(len(terms_only)-1):            
            for j in range(i+1, len(terms_only)):
                w1, w2 = sorted([terms_only[i], terms_only[j]])                
                if w1 != w2:
                    com[w1][w2] += 1

In [49]:
com_max = []
# For each term, look for the most common co-occurrent terms
for t1 in com:
    t1_max_terms = max(com[t1].items(), key=operator.itemgetter(1))[:10]
    for t2 in t1_max_terms:
        com_max.append(((t1, t2), com[t1][t2]))
# Get the most frequent co-occurrences
terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
print(terms_max[:10])

[((u'\ud83d', u'\ude21'), 13722), ((u'RT', u'Romo'), 3822), ((u'Another', u'RT'), 3564), ((u"Don't", u'RT'), 3528), ((u'interception', u'test'), 3528), ((u'Romo', u'Tony'), 3526), ((u'https://t.co/E8AEn8Uc6N', u'test'), 3521), ((u'Kuechly', u'Luke'), 3449), ((u'Luke', u'RT'), 3313), ((u'3', u'RT'), 1876)]


In [50]:
search_word = "Newton" # pass a search term
count_search = Counter()
with open(fname, 'r') as f:
    # f is the file pointer to the JSON data set
    for line in f:
        tweet = json.loads(line)
        try:
            terms_only = [term for term in preprocess(tweet['text']) if term not in stop and not term.startswith(('#', '@'))]
            if search_word in terms_only:
                count_search.update(terms_only)
        except:
            pass
print("Co-occurrence for %s:" % search_word)
print(count_search.most_common(20))

Co-occurrence for Newton:
[(u'Newton', 175), (u'Cam', 119), (u'\ud83d', 87), (u'\udd25', 69), (u'RT', 53), (u'Dallas', 24), (u'wear', 24), (u'vs', 24), (u'https://t.co/gsVy9JuMxC', 23), (u'dabbing', 23), (u'custom', 23), (u'cleats', 23), (u'I', 21), (u'Romo', 19), (u'Olsen', 19), (u'catch', 17), (u'challenging', 15), (u'Tony', 14), (u'Panthers', 11), (u'get', 10)]


In [51]:
#Plotting the most frequent words using D3.js and vincent
word_freq = terms_max[:20]
labels, freq = zip(*word_freq)
print labels, freq

((u'\ud83d', u'\ude21'), (u'RT', u'Romo'), (u'Another', u'RT'), (u"Don't", u'RT'), (u'interception', u'test'), (u'Romo', u'Tony'), (u'https://t.co/E8AEn8Uc6N', u'test'), (u'Kuechly', u'Luke'), (u'Luke', u'RT'), (u'3', u'RT'), (u'\ud83c', u'\ud83d'), (u'6', u'RT'), (u'2', u'RT'), (u'PICKKKKKKKKKKKKKKKKK', u'x2'), (u'That', u'defense'), (u'Tony', u'much'), (u'much', u'picking'), (u'liked', u'much'), (u'defense', u'much'), (u'picking', u'though')) (13722, 3822, 3564, 3528, 3528, 3526, 3521, 3449, 3313, 1876, 1730, 1722, 1693, 1215, 1210, 1209, 1204, 1204, 1204, 1203)


In [52]:
#Plot the top two words frequency as a bar chart
n_groups = 20

fig, ax = plt.subplots()

index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.4
error_config = {'ecolor': '0.3'}

rects1 = plt.bar(index, freq, bar_width,
                 alpha=opacity,
                 color='b',
                 error_kw=error_config,
                 label='Top 2-words together')

plt.xlabel('labels')
plt.ylabel('freq')
plt.xticks(index + bar_width, (labels))
ax.set_xticklabels( labels, rotation=90, va='center' ) ;
plt.legend()

plt.tight_layout()
plt.show()

In [56]:
def per_minute(SearchKeyword):
    dates_SearchKeyword = []
    # f is the file pointer to the JSON data set
    with open(fname, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            
            try:
                # let's focus on hashtags only at the moment
                terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('#')]
                # track when the hashtag is mentioned
                if '#' + SearchKeyword in terms_hash:
                    dates_SearchKeyword.append(tweet['created_at'])
            except:
                pass
            
    # a list of "1" to count the hashtags
    ones = [1]*len(dates_SearchKeyword)
    # the index of the series
    idx = pandas.DatetimeIndex(dates_SearchKeyword)
    # the actual series (at series of 1s for the moment)
    SearchKeyword = pandas.Series(ones, index=idx)

    # Resampling / bucketing
    per_minute_freq = SearchKeyword.resample('1Min', how='sum').fillna(0)
    return per_minute_freq

In [57]:
per_minute_CARvsDAL = per_minute('CARvsDAL')
per_minute_Cowboys = per_minute('Cowboys')
per_minute_Panthers = per_minute('Panthers')
per_minute_TOUCHDOWN = per_minute('TOUCHDOWN')

In [59]:
print per_minute_TOUCHDOWN.head()

2015-11-26 22:41:00     2
2015-11-26 22:42:00    12
2015-11-26 22:43:00     3
2015-11-26 22:44:00     1
2015-11-26 22:45:00     0
Freq: T, dtype: float64


In [25]:
ts = pandas.Series(per_minute_CARvsDAL)
plt.figure()
ts.plot()
plt.show()

In [58]:
# all the data together
match_data = dict(CARvsDAL=per_minute_CARvsDAL, Cowboys=per_minute_Cowboys, Panthers=per_minute_Panthers,
                  TOUCHDOWN=per_minute_TOUCHDOWN)
# we need a DataFrame, to accommodate multiple series
all_matches = pandas.DataFrame(data=match_data,
                               index=per_minute_CARvsDAL.index)
# Resampling as above
all_matches = all_matches.resample('1Min', how='sum').fillna(0)

columns_list = ['CARvsDAL','Cowboys','Panthers', 'TOUCHDOWN']
df = pandas.DataFrame(all_matches, columns=columns_list)
plt.figure()
df.plot()
plt.show()